ET will be used to get company specific and market/economic news.
Company names will be taken from the bse and nse scraper.

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException

First we will scrape company specific articles

In [ ]:
%store -r nse_data
%store -r bse_data
#combined list of companies containg only unique value
#convert each list to set to apply union operator
company_names=list(set(nse_data["company_names"]) | set(bse_data["company_names"]))
company_names[0:5]

In [ ]:
et_url = "https://economictimes.indiatimes.com/"
driver=webdriver.Chrome()
driver.get(et_url)
search_box=WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "ticker_newsearch")))
article_links=[]
for name in company_names:
    search_box.send_keys(name)
    time.sleep(3)
    result_items = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".searchListAll ul li")))
    result_items[1].click()

    WebDriverWait(driver, 30).until(EC.number_of_windows_to_be(2))
    first_window=driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle!=first_window:
            driver.switch_to.window(window_handle)
            break

    more_news_link = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".more_section a")))
    driver.execute_script("arguments[0].click();", more_news_link) # for some reason more_new_link.click() does not work, and driver.execute for result_items[1] does not work

    WebDriverWait(driver, 30).until(EC.number_of_windows_to_be(3))
    second_window=driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle!=second_window and window_handle!=first_window:
            driver.switch_to.window(window_handle)
            break

    def closeTabs():
        driver.close();
        driver.switch_to.window(second_window)
        driver.close()
        driver.switch_to.window(first_window)

    articles_container=WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".updates-container .all")))
    articles=articles_container[0].find_elements(By.CSS_SELECTOR, ".eachStory")
    if len(articles)==0:
        closeTabs()
        continue

    link=0
    for article in articles:
        print(link)
        link+=1
        a_tag=article.find_element(By.TAG_NAME, "a")
        href=a_tag.get_dom_attribute("href")
        #don't include pdfs
        if href[-3:]=="pdf":
            continue
        article_links.append(href)
    closeTabs()
driver.close()

In [ ]:
print(len(article_links))

now we will scrape articles on stock news, economy indicators and economy

In [4]:
def getLinks(url, links):
    driver=webdriver.Chrome()
    driver.get(url);
    last_height = driver.execute_script("return document.body.scrollHeight")
    #lazy loading, all articles were not loading, so scrolled
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    stock_news_articles=WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".tabdata .eachStory")))
    print(len(stock_news_articles))
    for article in stock_news_articles:
        #each story contains either span tag or a tag
        try:
            a_tag = article.find_element(By.TAG_NAME, "a")
            href = a_tag.get_attribute("href") 
        except NoSuchElementException:
            span_tag = article.find_element(By.TAG_NAME, "span")
            href = span_tag.get_attribute("href")
        if href[-3:]=="pdf":
            continue
        links.append(href)
    driver.close()

In [5]:
urls=[
    "https://economictimes.indiatimes.com/markets/stocks/news/",
    "https://economictimes.indiatimes.com/news/economy/indicators",
    "https://economictimes.indiatimes.com/news/economy/articlelist/1286551815.cms"
]
market_economic_news=[]
for i in range(3):
    getLinks(urls[i], market_economic_news)
print(len(market_economic_news))

48
48
48
144
